In [ ]:
!pip install blankly langchain langchain-community langchain-core langchain-google-genai -q

In [ ]:
!curl -sL https://github.com/klutometis/build-with-ai-data/archive/refs/heads/main.tar.gz | tar zxvf - --strip-components=1

build-with-ai-data-main/.git-crypt/
build-with-ai-data-main/.git-crypt/.gitattributes
build-with-ai-data-main/.git-crypt/keys/
build-with-ai-data-main/.git-crypt/keys/default/
build-with-ai-data-main/.git-crypt/keys/default/0/
build-with-ai-data-main/.git-crypt/keys/default/0/6C7816ECABE6500B1A4A1B380EFC2ADA85B9BA9C.gpg
build-with-ai-data-main/.gitattributes
build-with-ai-data-main/.gitmodules
build-with-ai-data-main/.pre-commit-config.yaml
build-with-ai-data-main/BUILD
build-with-ai-data-main/WORKSPACE
build-with-ai-data-main/backtest.json
build-with-ai-data-main/blankly.json
build-with-ai-data-main/data.json
build-with-ai-data-main/generate.py
build-with-ai-data-main/keys.json
build-with-ai-data-main/params.py
build-with-ai-data-main/pre-commit/
build-with-ai-data-main/requirements.in
build-with-ai-data-main/requirements.txt
build-with-ai-data-main/run.sh
build-with-ai-data-main/settings.json


In [ ]:
import ast
import json
import logging
import pprint
import random
import sys
import textwrap
from datetime import date, datetime
from enum import Enum

import blankly
import langchain
import numpy as np
import pandas as pd
from google.colab import userdata
from langchain.output_parsers.enum import EnumOutputParser
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain.tools import tool
from langchain.tools.render import render_text_description
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

TICKER = "TSLA"

logger = logging.getLogger("notebook")
logger.setLevel(logging.INFO)

In [ ]:
def plot(data, ticker):
    import plotly.express as px
    import plotly.graph_objects as go

    data["time"] = pd.to_datetime(data["time"], unit="s")

    coefficients = np.polyfit(data.index, data["Account Value (USD)"], deg=1)
    polynomial = np.poly1d(coefficients)

    fig = go.Figure()
    fig = fig.add_trace(
        go.Scatter(
            x=data["time"],
            y=data["Account Value (USD)"],
            mode="lines",
            name="Account Value (USD)",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=data["time"],
            y=polynomial(data.index),
            mode="lines",
            name="Regression",
        )
    )
    fig.update_layout(
        title=f"Time vs. money ({ticker})",
        xaxis_title="Time",
        yaxis_title="USD",
        yaxis=dict(
            tickprefix="$", ticks="outside"
        ),  # Add dollar sign as prefix
        yaxis_tickformat="s",
        width=800,  # Set the width of the plot
        height=600,  # Set the height of the plot
        legend=dict(
            x=0.2,  # Legend x position (0 (left) to 1 (right) within the plotting area)
            y=0.9,  # Legend y position (0 (bottom) to 1 (top) and above)
            xanchor="center",  # Anchor the x position of the legend
            bgcolor="rgba(255, 255, 255, 0.5)",  # Semi-transparent white background
            bordercolor="Black",
            borderwidth=0,
        ),
    )
    fig.show()

In [ ]:
class Action(Enum):
    BUY = 1
    HOLD = 2
    SELL = 3


def init(symbol, state: blankly.StrategyState):
    pass


def price_event(price, symbol, state: blankly.StrategyState):
    # TODO(danenberg): Replace this with a call to LLM.
    match random.choices(list(Action), weights=[1, 1, 1], k=1)[0]:
        case Action.BUY:
            if state.interface.cash >= price:
                state.interface.market_order(symbol, side="buy", size=1)
        case Action.SELL:
            if state.interface.account[TICKER]["available"]:
                state.interface.market_order(symbol, side="sell", size=1)

    logger.info(f"{state.interface.account[TICKER]=}")


exchange = blankly.Alpaca()
strategy = blankly.Strategy(exchange)
strategy.add_price_event(
    price_event, symbol=TICKER, resolution="1w", init=init
)

results = strategy.backtest(to="1y", initial_values={"USD": 10000})
data = results.get_account_history()

INFO: No portfolio name to load specified, defaulting to the first in the file: (example-portfolio). This is fine if there is only one portfolio in use.
INFO:notebook:state.interface.account[TICKER]={'available': 0, 'hold': 0}



Backtesting...


INFO:notebook:state.interface.account[TICKER]={'available': 1.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 2.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 1.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 1.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 1.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 1.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:noteboo

Progress: [##########] 100% Done...


In [ ]:
plot(data, "TSLA")

In [ ]:
exchange = blankly.Alpaca()
strategy = blankly.Strategy(exchange)
strategy.add_price_event(
    consult_gemini, symbol=TICKER, resolution="1w", init=init
)

results = strategy.backtest(to="12w", initial_values={"USD": 10000})
data = results.get_account_history()